# 如何添加消息历史

:::info 前提条件

本指南假定您熟悉以下概念：

- [链式运行](/docs/how_to/sequence/)
- [提示模板](/docs/concepts/prompt_templates)
- [聊天消息](/docs/concepts/messages)

:::

```{=mdx}
:::note

本指南之前介绍了 [RunnableWithMessageHistory](https://api.js.langchain.com/classes/_langchain_core.runnables.RunnableWithMessageHistory.html) 抽象。您可以在 [v0.2 文档](https://js.langchain.com/v0.2/docs/how_to/message_history/) 中查看该版本的指南。

LangGraph 实现相较于 `RunnableWithMessageHistory` 提供了许多优势，包括持久化应用程序状态的任意组件的能力（而不仅仅是消息）。

:::
```


在构建聊天机器人时，将对话状态传递到链中并从中取出至关重要。LangGraph 实现了一个内置的持久化层，允许链状态自动在内存中或外部后端（如 SQLite、Postgres 或 Redis）中持久化。详细信息可以在 LangGraph 持久化文档中找到。

在本指南中，我们演示了如何通过将任意 LangChain 可运行对象包装在一个最小的 LangGraph 应用程序中来添加持久化功能。这使我们能够持久化消息历史和链状态的其他元素，简化多轮应用程序的开发。它还支持多个线程，使单个应用程序可以分别与多个用户进行交互。

## 准备工作

```{=mdx}
import Npm2Yarn from "@theme/Npm2Yarn";

<Npm2Yarn>
  @langchain/core @langchain/langgraph
</Npm2Yarn>
```

我们还设置一个聊天模型，用于以下示例。

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [30]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
});

## 示例：消息输入

为[聊天模型](/docs/concepts/chat_models)添加记忆提供了一个简单的示例。聊天模型接受消息列表作为输入，并输出一条消息。LangGraph 包含一个内置的 `MessagesState`，我们可以为此目的使用它。

下面，我们：
1. 将图状态定义为消息列表；
2. 向图中添加一个调用聊天模型的节点；
3. 使用内存检查点存储运行之间的消息来编译该图。

:::info

LangGraph 应用程序的输出是其[状态](https://langchain-ai.github.io/langgraphjs/concepts/low_level/)。

:::

In [31]:
import { START, END, MessagesAnnotation, StateGraph, MemorySaver } from "@langchain/langgraph";

// Define the function that calls the model
const callModel = async (state: typeof MessagesAnnotation.State) => {
  const response = await llm.invoke(state.messages);
  // Update message history with response:
  return { messages: response };
};

// Define a new graph
const workflow = new StateGraph(MessagesAnnotation)
  // Define the (single) node in the graph
  .addNode("model", callModel)
  .addEdge(START, "model")
  .addEdge("model", END);

// Add memory
const memory = new MemorySaver();
const app = workflow.compile({ checkpointer: memory });

运行应用程序时，我们会传入一个指定`thread_id`的配置对象。该ID用于区分对话线程（例如，不同用户之间的对话）。

In [32]:
import { v4 as uuidv4 } from "uuid";

const config = { configurable: { thread_id: uuidv4() } }

然后我们可以调用该应用程序：

In [33]:
const input = [
  {
    role: "user",
    content: "Hi! I'm Bob.",
  }
]
const output = await app.invoke({ messages: input }, config)
// The output contains all messages in the state.
// This will log the last message in the conversation.
console.log(output.messages[output.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABTqCeKnMQmG9IH8dNF5vPjsgXtcM",
  "content": "Hi Bob! How can I assist you today?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 10,
      "promptTokens": 12,
      "totalTokens": 22
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_e375328146"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 12,
    "output_tokens": 10,
    "total_tokens": 22
  }
}


In [34]:
const input2 = [
  {
    role: "user",
    content: "What's my name?",
  }
]
const output2 = await app.invoke({ messages: input2 }, config)
console.log(output2.messages[output2.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABTqD5jrJXeKCpvoIDp47fvgw2OPn",
  "content": "Your name is Bob. How can I help you today, Bob?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 14,
      "promptTokens": 34,
      "totalTokens": 48
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_e375328146"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 34,
    "output_tokens": 14,
    "total_tokens": 48
  }
}


请注意，不同线程的状态是相互分离的。如果我们使用新的 `thread_id` 向线程发出相同的查询，模型会表示它不知道答案：

In [35]:
const config2 = { configurable: { thread_id: uuidv4() } }
const input3 = [
  {
    role: "user",
    content: "What's my name?",
  }
]
const output3 = await app.invoke({ messages: input3 }, config2)
console.log(output3.messages[output3.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABTqDkctxwmXjeGOZpK6Km8jdCqdl",
  "content": "I'm sorry, but I don't have access to personal information about users. How can I assist you today?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 21,
      "promptTokens": 11,
      "totalTokens": 32
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_52a7f40b0b"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 11,
    "output_tokens": 21,
    "total_tokens": 32
  }
}


## 示例：对象输入

LangChain 可运行对象通常通过单个对象参数中不同的键接受多个输入。一个常见的例子是带有多个参数的提示模板。

之前我们的可运行对象是一个聊天模型，这里我们将提示模板和聊天模型串联在一起。

In [36]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "Answer in {language}."],
  new MessagesPlaceholder("messages"),
])

const runnable = prompt.pipe(llm);

在此场景中，我们定义图状态包含这些参数（除了消息历史）。

请注意以下状态中的定义：
- 对 `messages` 列表的更新会追加消息；
- 对 `language` 字符串的更新会覆盖该字符串。

In [37]:
import { START, END, StateGraph, MemorySaver, MessagesAnnotation, Annotation } from "@langchain/langgraph";

// Define the State
// highlight-next-line
const GraphAnnotation = Annotation.Root({
  // highlight-next-line
  language: Annotation<string>(),
  // Spread `MessagesAnnotation` into the state to add the `messages` field.
  // highlight-next-line
  ...MessagesAnnotation.spec,
})


// Define the function that calls the model
const callModel2 = async (state: typeof GraphAnnotation.State) => {
  const response = await runnable.invoke(state);
  // Update message history with response:
  return { messages: [response] };
};

const workflow2 = new StateGraph(GraphAnnotation)
  .addNode("model", callModel2)
  .addEdge(START, "model")
  .addEdge("model", END);

const app2 = workflow2.compile({ checkpointer: new MemorySaver() });

In [38]:
const config3 = { configurable: { thread_id: uuidv4() } }
const input4 = {
  messages: [
    {
      role: "user",
      content: "What's my name?",
    }
  ],
  language: "Spanish",
} 
const output4 = await app2.invoke(input4, config3)
console.log(output4.messages[output4.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABTqFnCASRB5UhZ7XAbbf5T0Bva4U",
  "content": "Lo siento, pero no tengo suficiente información para saber tu nombre. ¿Cómo te llamas?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 19,
      "promptTokens": 19,
      "totalTokens": 38
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_e375328146"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 19,
    "output_tokens": 19,
    "total_tokens": 38
  }
}


## 管理消息历史记录
消息历史记录（以及应用程序状态的其他元素）可以通过 `.getState` 访问：

In [39]:
const state = (await app2.getState(config3)).values

console.log(`Language: ${state.language}`);
console.log(state.messages)

Language: Spanish
[
  HumanMessage {
    "content": "What's my name?",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "id": "chatcmpl-ABTqFnCASRB5UhZ7XAbbf5T0Bva4U",
    "content": "Lo siento, pero no tengo suficiente información para saber tu nombre. ¿Cómo te llamas?",
    "additional_kwargs": {},
    "response_metadata": {
      "tokenUsage": {
        "completionTokens": 19,
        "promptTokens": 19,
        "totalTokens": 38
      },
      "finish_reason": "stop",
      "system_fingerprint": "fp_e375328146"
    },
    "tool_calls": [],
    "invalid_tool_calls": []
  }
]


我们还可以通过 `.updateState` 来更新状态。例如，我们可以手动添加一条新消息：

In [40]:
const _ = await app2.updateState(config3, { messages: [{ role: "user", content: "test" }]})

In [41]:
const state2 = (await app2.getState(config3)).values

console.log(`Language: ${state2.language}`);
console.log(state2.messages)

Language: Spanish
[
  HumanMessage {
    "content": "What's my name?",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "id": "chatcmpl-ABTqFnCASRB5UhZ7XAbbf5T0Bva4U",
    "content": "Lo siento, pero no tengo suficiente información para saber tu nombre. ¿Cómo te llamas?",
    "additional_kwargs": {},
    "response_metadata": {
      "tokenUsage": {
        "completionTokens": 19,
        "promptTokens": 19,
        "totalTokens": 38
      },
      "finish_reason": "stop",
      "system_fingerprint": "fp_e375328146"
    },
    "tool_calls": [],
    "invalid_tool_calls": []
  },
  HumanMessage {
    "content": "test",
    "additional_kwargs": {},
    "response_metadata": {}
  }
]


有关管理状态（包括删除消息）的详细信息，请参阅 LangGraph 文档：

- [如何删除消息](https://langchain-ai.github.io/langgraphjs/how-tos/delete-messages/)
- [如何查看和更新过去的图状态](https://langchain-ai.github.io/langgraphjs/how-tos/time-travel/)